In [1]:
#inport python libraries
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import LabelEncoder

In [2]:
#load datasets
accounts = pd.read_csv("accounts.csv")
products = pd.read_csv("products.csv")
sales_pipelines = pd.read_csv("sales_pipeline.csv")
sales_teams = pd.read_csv("sales_teams.csv")

In [3]:
#merge all of the datasets
merged = pd.merge(sales_pipelines, accounts, on="account", how="left")
merged = pd.merge(merged, products, on="product", how="left")
merged = pd.merge(merged, sales_teams, on="sales_agent", how="left")

In [4]:
#drop columns
for col in ["opportunity_id", "subsidiary_of"]:
    if col in merged.columns:
        merged.drop(columns=col, inplace=True)

#drop the duplicate columns in the merged dataset
merged.drop_duplicates(inplace=True)

#standardize the product name
merged['product'] = merged['product'].replace("GTXPro", "GTX Pro")

In [5]:
#convert date columns
if "engage_date" in merged.columns:
    merged["engage_date"] = pd.to_datetime(merged["engage_date"], errors="coerce")
if "close_date" in merged.columns:
    merged["close_date"] = pd.to_datetime(merged["close_date"], errors="coerce")

#define categorical and numerical columns
categorical_columns = merged.select_dtypes(include="object").columns
numerical_columns = merged.select_dtypes(include="number").columns

#handle missing values for categorical variables with mode imputation
for col in categorical_columns:
    if merged[col].isna().sum() > 0:
        merged[col] = merged[col].fillna(merged[col].mode()[0])

#handle missing values for numerical variables with median imputation
for col in numerical_columns:
    if col not in ["close_value", "close_date", "engage_date"]: #not including null values since the deal has not been closed
        merged[col] = merged[col].fillna(merged[col].median())

#handle outliers (except close_value)
for col in numerical_columns:
    if col != "close_value":
        Q1 = merged[col].quantile(0.25)
        Q3 = merged[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_side = Q1 - 1.5 * IQR
        upper_side = Q3 + 1.5 * IQR
        merged[col] = merged[col].clip(lower=lower_side, upper=upper_side)

In [6]:
#apply log transformation
cols_to_log = [c for c in ["close_value", "revenue", "employees"] if c in merged.columns]
for col in cols_to_log:
    merged[col + "_log"] = np.log1p(merged[col])

#encode categorical columns
for col in categorical_columns:
    le = LabelEncoder()
    merged[col] = le.fit_transform(merged[col])
merged.head(10)

,sales_agent,product,account,deal_stage,engage_date,close_date,close_value,sector,year_established,revenue,employees,office_location,series,sales_price,manager,regional_office,close_value_log,revenue_log,employees_log
0,20,2,8,3,2016-10-20,2017-03-01,1054.0,5,2001.0,718.62,2448.0,14,1,1096.0,2,0,6.961296,6.578723,7.803435
1,6,4,37,3,2016-10-25,2017-03-11,4514.0,4,2002.0,3178.24,4540.0,14,1,1096.0,3,0,8.415160,8.064397,8.420903
2,6,6,8,3,2016-10-25,2017-03-07,50.0,5,2001.0,718.62,2448.0,14,2,55.0,3,0,3.931826,6.578723,7.803435
3,20,1,10,3,2016-10-25,2017-03-09,588.0,7,1998.0,2714.90,2641.0,14,1,550.0,2,0,6.378426,7.906879,7.879291
4,29,1,33,3,2016-10-25,2017-03-02,517.0,6,1982.0,792.46,1299.0,14,1,550.0,5,2,6.249975,6.676403,7.170120
5,0,6,57,3,2016-10-29,2017-03-01,49.0,4,1992.0,3922.42,6837.0,14,2,55.0,2,0,3.912023,8.274719,8.830251
6,26,6,39,3,2016-10-30,2017-03-02,57.0,5,1989.0,1388.67,3583.0,14,2,55.0,1,2,4.060443,7.236822,8.184235
7,17,1,9,3,2016-11-01,2017-03-07,601.0,1,1993.0,4269.90,6472.0,14,1,550.0,1,2,6.400257,8.359580,8.775395
8,21,2,83,3,2016-11-01,2017-03-03,1026.0,4,1984.0,441.08,1210.0,14,1,1096.0,3,0,6.934397,6.091491,7.099202
9,12,5,34,0,2016-11-03,NaT,NaN,5,1995.0,1698.20,3492.0,14,2,3393.0,5,2,NaN,7.437913,8.158516


In [7]:
#implement feature engineering

#date/timeline features
if "engage_date" in merged.columns and "close_date" in merged.columns:
    #time between engagement and closing
    merged["days_to_close"] = (merged["close_date"] - merged["engage_date"]).dt.days

    #flag for deals closed within 30 days
    merged["closed_within_30d"] = merged["days_to_close"].apply(lambda x: 1 if pd.notnull(x) and x <= 30 else 0)

    #month and quarter features
    merged["engage_month"] = merged["engage_date"].dt.month
    merged["engage_quarter"] = merged["engage_date"].dt.quarter
    merged["close_month"] = merged["close_date"].dt.month
    merged["close_quarter"] = merged["close_date"].dt.quarter

#interaction count per account
if "account" in merged.columns:
    account_interaction_counts = merged.groupby("account").size().rename("account_interaction_count")
    merged = merged.merge(account_interaction_counts, on="account", how="left")

#num of deals closed by agent
if "sales_agent" in merged.columns and "close_date" in merged.columns:
    agent_closed_deals = merged[merged["close_date"].notnull()].groupby("sales_agent").size().rename("agent_closed_deals")
    merged = merged.merge(agent_closed_deals, on="sales_agent", how="left")
    merged["agent_closed_deals"] = merged["agent_closed_deals"].fillna(0)

#opportunity win rate by account
if "account" in merged.columns and "close_value" in merged.columns:
    merged["won_deal"] = merged["close_value"].notnull().astype(int)
    win_rate = merged.groupby("account")["won_deal"].mean().rename("account_win_rate")
    merged = merged.merge(win_rate, on="account", how="left")

#avg deal size by product
if "product" in merged.columns and "close_value" in merged.columns:
    avg_deal_size = merged.groupby("product")["close_value"].mean().rename("avg_deal_size_by_product")
    merged = merged.merge(avg_deal_size, on="product", how="left")

In [8]:
# Summary statistics for numerical features
display(merged.describe().T)

# Summary statistics for categorical features however, only if any exist
cat_cols = merged.select_dtypes(include='object').columns
if len(cat_cols) > 0:
    display(merged[cat_cols].describe().T)
else:
    print("No categorical columns to summarize.")

#evaluate features most correlated with close_value (deals completed)
if "close_value" in merged.columns:
    corr = merged.corr(numeric_only=True)["close_value"].sort_values(ascending=False)
    display(corr)

,count,mean,min,25%,50%,75%,max,std
sales_agent,8448.0,13.954664,0.0,6.0,14.0,21.0,29.0,8.693372
product,8448.0,3.444602,0.0,2.0,4.0,5.0,6.0,1.823452
account,8448.0,40.109848,0.0,23.0,34.0,58.0,84.0,22.398814
deal_stage,8448.0,1.84813,0.0,1.0,3.0,3.0,3.0,1.221922
engage_date,8235,2017-06-14 03:13:02.950819840,2016-10-20 00:00:00,2017-04-03 00:00:00,2017-06-23 00:00:00,2017-08-27 00:00:00,2017-12-27 00:00:00,NaN
close_date,6711,2017-08-01 03:32:25.641484288,2017-03-01 00:00:00,2017-05-18 00:00:00,2017-08-02 00:00:00,2017-10-18 00:00:00,2017-12-31 00:00:00,NaN
close_value,6711.0,1490.915512,0.0,0.0,472.0,3225.0,30288.0,2320.670773
sector,8448.0,4.95348,0.0,3.0,5.0,7.0,9.0,2.351616
year_established,8448.0,1995.409624,1979.0,1989.0,1995.0,2000.0,2016.5,8.555363
revenue,8448.0,2138.873129,4.54,702.72,1698.2,2871.35,6124.295,1807.511943


No categorical columns to summarize.


close_value                  1.000000
close_value_log              0.703194
avg_deal_size_by_product     0.679986
deal_stage                   0.490799
sales_price                  0.482187
regional_office              0.056868
days_to_close                0.049888
employees_log                0.038439
revenue_log                  0.036576
employees                    0.032586
revenue                      0.029614
account_win_rate             0.027586
manager                      0.026562
product                      0.025330
account                      0.025197
office_location              0.020498
agent_closed_deals           0.019563
close_month                 -0.003534
engage_month                -0.011086
sector                      -0.017211
engage_quarter              -0.018734
year_established            -0.025345
account_interaction_count   -0.035372
close_quarter               -0.036586
sales_agent                 -0.047883
closed_within_30d           -0.049249
series      